# Necessary Imports

In [1]:
pip install transformers torch onnx openvino-dev[onnx] datasets accelerate openvino peft bitsandbytes trl -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 956.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 22.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Usi

In [1]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.44.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: e:\coding material\python pr\research\gpt-neo\.venv\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl
Name: accelerate
Version: 0.33.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: e:\coding material\python pr\research\gpt-neo\.venv\lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
  Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)


In [1]:
import pandas as pd
import torch
from datasets import Dataset
from random import randrange
#from peft import Loraconfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load and Save Pre-trained Model and Tokenizer

In [2]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m") #TinyLlama/TinyLlama-1.1B-Chat-v1.0, EleutherAI/gpt-neo-125m, Qwen/Qwen2-0.5B-Instruct
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

model.save_pretrained("./my_gpt_model")
tokenizer.save_pretrained("./my_gpt_model")

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shiva\.cache\huggingface\hub\models--EleutherAI--gpt-neo-125m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


('./my_gpt_model\\tokenizer_config.json',
 './my_gpt_model\\special_tokens_map.json',
 './my_gpt_model\\vocab.json',
 './my_gpt_model\\merges.txt',
 './my_gpt_model\\added_tokens.json',
 './my_gpt_model\\tokenizer.json')

# Convert PyTorch Model to ONNX Format

In [3]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./my_gpt_model")
model = AutoModelForCausalLM.from_pretrained("./my_gpt_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "gptneo_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Model converted to ONNX format successfully.")

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\models\gpt_neo\modeling_gpt_neo.py:216: 

Model converted to ONNX format successfully.


# Convert ONNX Model to OpenVINO IR Format

In [4]:
import openvino as ov
import os

ir_model = ov.convert_model("gptneo_model.onnx",
                            input=[ov.PartialShape([1, 4])])

ov.save_model(ir_model, "gptneo_ir_model.xml")

print("Model converted to IR format successfully.")

print("XML file exists:", os.path.exists("gptneo_ir_model.xml"))
print("BIN file exists:", os.path.exists("gptneo_ir_model.bin"))

Model converted to IR format successfully.
XML file exists: True
BIN file exists: True


# Pre Processing Dataset

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('medquad.csv')

df = df[['question', 'answer']]

In [3]:
train_val_df, sample_df = train_test_split(df, test_size=0.1, random_state=42) #using only sample df as the whole dataset is very big

In [4]:
sample_df['text'] = 'Question:\n' + sample_df['question'] + '\n\nAnswer:\n' + sample_df['answer']
sample_df.drop(columns=['question','answer'], axis=1,inplace=True)
# sample_df.to_csv('/content/pre_dataset.csv', index=False)

In [5]:
df = pd.read_csv('pre_dataset.csv')

train = Dataset.from_pandas(df)
train_dataset_size = len(train)
print(train_dataset_size)

1642


In [6]:
train

Dataset({
    features: ['text'],
    num_rows: 1642
})

# Load and Prepare Model, Tokenizer for finetuning

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, DatasetDict
import torch

model_name = "./my_gpt_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

train_dataset_size = len(train)
batch_size = 4
steps_per_epoch = train_dataset_size // batch_size

target_steps = 300
num_epochs = max(3, target_steps // steps_per_epoch)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
)

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    dataset_text_field='text',
    max_seq_length=1042,
    tokenizer=tokenizer,
    packing=True,
    callbacks=[PrinterCallback()]
)

trainer.train()

trainer.save_model("./fine_tuned_gpt_model")

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\trl\trainer\sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\trl\trainer\sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\trl\trainer\sft_tra

Step 10: Training Loss: 2.27120
{'loss': 2.2712, 'grad_norm': 3.3194222450256348, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}


  3%|▎         | 20/771 [30:39<28:27:57, 136.45s/it]

Step 20: Training Loss: 2.26500
{'loss': 2.265, 'grad_norm': 4.622042655944824, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.08}


  4%|▍         | 30/771 [38:40<10:28:57, 50.93s/it] 

...

Step 760: Training Loss: 1.25760
{'loss': 1.2576, 'grad_norm': 2.2671334743499756, 'learning_rate': 2.029520295202952e-06, 'epoch': 2.96}


100%|█████████▉| 770/771 [11:44:14<00:22, 22.97s/it]

Step 770: Training Loss: 1.33300
{'loss': 1.333, 'grad_norm': 1.8313028812408447, 'learning_rate': 1.845018450184502e-07, 'epoch': 3.0}


100%|██████████| 771/771 [11:44:38<00:00, 54.84s/it]

Step 771: Logs: {'train_runtime': 42278.1228, 'train_samples_per_second': 0.036, 'train_steps_per_second': 0.018, 'total_flos': 819722969837568.0, 'train_loss': 1.630171654443951, 'epoch': 3.0}
{'train_runtime': 42278.1228, 'train_samples_per_second': 0.036, 'train_steps_per_second': 0.018, 'train_loss': 1.630171654443951, 'epoch': 3.0}


# Exporting the Fine-Tuned GPT-Neo Model to ONNX Format

In [8]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "fine_tuned_gptneo_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Fine-tuned model converted to ONNX format successfully.")

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\models\gpt_neo\modeling_gpt_neo.py:216: 

Fine-tuned model converted to ONNX format successfully.


# Converting the Fine-Tuned GPT-Neo Model to OpenVINO IR Format

In [9]:
from openvino.tools import mo

ov_model = mo.convert_model("fine_tuned_gptneo_model.onnx",
                            input=[('input_ids', [1, -1]), ('attention_mask', [1, -1])],
                            output=['output'])

from openvino.runtime import serialize
serialize(ov_model, "fine_tuned_gptneo_ir_model.xml")

print("Fine-tuned model converted to IR format successfully.")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Fine-tuned model converted to IR format successfully.


# OpenVINO IR Format Fine Tuned Model Inference

In [10]:
import openvino.runtime as ov
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")

core = ov.Core()
model = core.read_model("fine_tuned_gptneo_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")

infer_request = compiled_model.create_infer_request()

def generate_text(prompt, max_length=200): # The maximum length is 200; the answer may be short.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]

        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

prompt = "What is (are) Schwannomatosis ?"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)

#True answer: Schwannomatosis is a rare form of neurofibromatosis that is primarily characterized by multiple schwannomas (benign tumors of the nervous system) in the absence of bilateral (affecting both sides) vestibular schwannomas. Signs and symptoms of the condition vary based on the size, location and number of schwannomas but may include pain; numbness; tingling; and/or weakness in the fingers and toes. Inherited forms of the disorder account for only 15 percent of all cases. In some of these families, schwannomatosis is caused by changes (mutations) in the SMARCB1 or LZTR1 genes; in other cases, the exact underlying cause is unknown. When inherited, the condition is passed down in an autosomal dominant manner with highly variable expressivity and reduced penetrance. Treatment is based on the signs and symptoms present in each person but may include medications and/or surgery.

Generated text: What is (are) Schwannomatosis?

Answer:
Schwannomatosis is a rare condition that affects the nervous system. The exact cause is unknown, but it is believed to be a disorder of the nervous system. The signs and symptoms of this condition are similar to those of other connective tissue disorders, such as rheumatoid arthritis and multiple sclerosis.
                
Signs and Symptoms
                
Signs and symptoms of Schwannomatosis include:
                
A lump in the back of the neck. The lump can be painful and hard to move. The lump can also be painful and hard to move.
                
A lump in the back of the neck. The lump can be


# Simple Fine Tuned Model Inference

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

fine_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")
fine_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt_model")

In [16]:
# fine_model.cuda() # Using GPU for generating text

prompt = "What are the symptoms of Crome syndrome ?"

inputs = fine_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs.input_ids.to(device)
attention_mask = inputs.attention_mask.to(device)

outputs = fine_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=200,
    temperature=0.6,
    pad_token_id=fine_tokenizer.eos_token_id
)

e:\Coding Material\Python Pr\Research\GPT-Neo\.venv\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [17]:
generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

What are the symptoms of Crome syndrome?

Answer:
What are the signs and symptoms of Crome syndrome? The Human Phenotype Ontology provides the following list of signs and symptoms for Crome syndrome. If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Abnormality of the skin - Abnormality of the fingernails - Abnormality of the palate - Abnormality of the palate - Abnormality of the ribs - Abnormality of the tongue - Abnormality of the tongue - Abnormality of the testes - Abnormality of the testes - Abnormality of the testes - Abnormality of the testes - Abnormality of the testes - Abnormality of the testes - Abnormality of the testes - Abnormality of the
